# Proyecto final (CNN)

Omar David Hernández Aguirre  | A01383543  
Bernardo García Zermeño       | A00570682  
06 de junio de 2023  

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision.transforms import ToTensor
import os
import pandas as pd
import numpy as np

In [9]:
print(os.listdir('./archive/'))

['Test', 'Test.csv', 'Meta', 'Train', 'Meta.csv', 'Train.csv']


In [34]:
# Reading train and test data from CSVs
training_data = pd.read_csv('./archive/Train.csv') 
test_data = pd.read_csv('./archive/Test.csv')

In [45]:
# Transforming train data to tensor
tensor_training = training_data.drop(columns="Path")
tensor_training = torch.Tensor(tensor_training.values)
print(tensor_training)

tensor([[27., 26.,  5.,  ..., 22., 20., 20.],
        [28., 27.,  5.,  ..., 23., 22., 20.],
        [29., 26.,  6.,  ..., 24., 21., 20.],
        ...,
        [58., 62.,  5.,  ..., 53., 57., 42.],
        [63., 69.,  5.,  ..., 58., 63., 42.],
        [68., 69.,  7.,  ..., 62., 63., 42.]])


In [44]:
# Transforming test data to tensor
tensor_test = test_data.drop(columns="Path")
tensor_test = torch.Tensor(tensor_test.values)
print(tensor_test)

tensor([[53., 54.,  6.,  ..., 48., 49., 16.],
        [42., 45.,  5.,  ..., 36., 40.,  1.],
        [48., 52.,  6.,  ..., 43., 47., 38.],
        ...,
        [29., 29.,  6.,  ..., 24., 24.,  6.],
        [48., 49.,  5.,  ..., 43., 44.,  7.],
        [32., 31.,  6.,  ..., 27., 26., 10.]])
